In [ ]:
import nibabel as nib
import numpy as np

from nilearn.input_data import NiftiMasker
from nilearn.image import resample_to_img, math_img

from scipy.stats import spearmanr, rankdata

from sklearn.linear_model import LinearRegression
import os
import pickle
from collections import defaultdict

In [2]:
run_start_indices = [0, 267, 492, 812, 1137, 1373]

def residualize(X, confounds):
    """Vectorized residualization of fMRI data against confounds."""
    # Add intercept
    confounds_aug = np.hstack([confounds, np.ones((confounds.shape[0], 1))])
    betas, _, _, _ = np.linalg.lstsq(confounds_aug, X, rcond=None)
    predicted = confounds_aug @ betas
    residuals = X - predicted
    return residuals

def cross_validated_residualize(fmri_list, confound_list):
    """
    Residualizes each run using confound model trained on other runs.
    Returns a new list of residualized runs.
    """
    residualized_runs = []
    for i in range(len(fmri_list)):
        # Prepare training confounds and fMRI
        train_confounds = np.vstack([confound_list[j] for j in range(len(fmri_list)) if j != i])
        train_fmri = np.vstack([fmri_list[j] for j in range(len(fmri_list)) if j != i])

        # Fit model
        confounds_aug = np.hstack([train_confounds, np.ones((train_confounds.shape[0], 1))])
        betas, _, _, _ = np.linalg.lstsq(confounds_aug, train_fmri, rcond=None)

        # Apply to test confounds
        test_confounds = confound_list[i]
        test_confounds_aug = np.hstack([test_confounds, np.ones((test_confounds.shape[0], 1))])
        predicted = test_confounds_aug @ betas
        residuals = fmri_list[i] - predicted
        residualized_runs.append(residuals)

    return residualized_runs

def split_runs(data, indices, spliti):
    """
    Splits the data into multiple runs based on the indices and the specified dimension.
    
    Parameters:
    - data: The input data to be split (can be a 4D array).
    - indices: List of indices marking where the runs start.
    - spliti: The axis (dimension) to split the data along. For fMRI data, spliti would typically be 3 for time.
    
    Returns:
    - runs: A list containing the data split by runs.
    """
    runs = []
    for i in range(len(indices) - 1):
        start = indices[i]
        end = indices[i + 1]
        
        # Dynamically slice the data along the specified dimension (spliti)
        # Using slicing: data[... , start:end] where `spliti` defines which axis is sliced
        slices = [slice(None)] * data.ndim  # Create a list of slices to cover all dimensions
        slices[spliti] = slice(start, end)  # Update the slice for the specified dimension
        
        run_data = data[tuple(slices)]  # Apply the slice to the data
        runs.append(run_data)

    # The last run should end at the last time point
    start = indices[-1]
    slices = [slice(None)] * data.ndim  # Again, create a list of slices for all dimensions
    slices[spliti] = slice(start, None)  # Slice from the last start index to the end
    run_data = data[tuple(slices)]
    runs.append(run_data)

    return runs

def first_order_similarity(fmri):
    L2a = []  # List to store fMRI correlations

    num_timepoints = fmri.shape[0]

    for t1 in range(num_timepoints):
        for t2 in range(t1 + 1, num_timepoints):  # Upper-triangular matrix only (t2 > t1)
            
            # fMRI correlation: Compute correlation between voxel activity at t1 and t2
            fmri_corr = spearmanr(fmri[t1, :], fmri[t2, :]).correlation
            L2a.append(fmri_corr)

    return L2a

def mask_fmri_data(fmri_img, original_mask, roi=None):
    if roi=="body" or roi=="object" or roi=="scene" or roi=="face":
        mask_data = original_mask.get_fdata()

        # Determine dynamic threshold (≥ 60% of subjects)
        max_subjects_plus_1 = np.max(mask_data)
        estimated_n_subjects = int(max_subjects_plus_1 - 1)
        threshold = int(np.ceil(0.6 * estimated_n_subjects)) + 1  # +1 for the +1 encoding

        # Create binary mask
        binary_mask_data = (mask_data >= threshold).astype(np.uint8)
        voxel_count = np.count_nonzero(binary_mask_data)

        if voxel_count == 0:
            raise ValueError(f"No voxels meet the 60% threshold ({threshold}). Mask is empty.")

        # Create binary mask NIfTI
        binary_mask_img = nib.Nifti1Image(binary_mask_data, affine=original_mask.affine, header=original_mask.header)

        # Resample to fMRI image
        mask_resampled = resample_to_img(binary_mask_img, fmri_img, interpolation="nearest", force_resample=True, copy_header=True)

        # Apply the mask
        masker = NiftiMasker(mask_img=mask_resampled)
        masked_data = masker.fit_transform(fmri_img)
    elif roi==2:
        mask_img = math_img("np.logical_or(img == 1, img == 2)", img=original_mask)
        mask_resampled = resample_to_img(mask_img, fmri_img, interpolation="nearest", force_resample=True, copy_header=True)

        # Apply mask
        masker = NiftiMasker(mask_img=mask_resampled)
        masked_data = masker.fit_transform(fmri_img)

    else:
        mask_img = math_img(f"img == {roi}", img=original_mask)
        mask_resampled = resample_to_img(mask_img, fmri_img, interpolation="nearest", force_resample=True, copy_header=True)

        # Apply mask
        masker = NiftiMasker(mask_img=mask_resampled)
        masked_data = masker.fit_transform(fmri_img)


    return masked_data

def return_ranks_voxels(fmri_list, model_list):
    """
    Rank the correlations of the voxels to the model using vectorized Spearman correlation
    or negative absolute distance for 1D models.

    Args:
    - fmri_list: List of 5 NumPy arrays (each run's fMRI data of shape [timepoints, voxels])
    - model_list: List of 5 NumPy arrays (each run's model time series, shape [timepoints, features] or [timepoints])

    Returns:
    - top_voxel_indices: Numpy array of the indices of the top 10% highest-ranking voxels.
    """
    num_runs = len(fmri_list)
    num_voxels = fmri_list[0].shape[1]
    voxel_ranks = np.zeros((num_runs, num_voxels))

    for i in range(num_runs):
        train_fmri = fmri_list[i]       # Shape: (T, V)
        train_model = model_list[i]     # Shape: (T, F) or (T,)

        if train_model.ndim == 1:
            # 1D case — use negative absolute distance
            model_ts = rankdata(train_model) - np.mean(rankdata(train_model))  # rank and zero-mean
            fmri_ranked = np.apply_along_axis(rankdata, 0, train_fmri) - np.mean(train_fmri, axis=0)

            # Negative absolute difference per voxel
            avg_neg_abs_dist = -np.mean(np.abs(fmri_ranked - model_ts[:, None]), axis=0)
            voxel_ranks[i, :] = rankdata(-avg_neg_abs_dist, method='average')

        else:
            # 2D case — Spearman correlation
            ranked_fmri = np.apply_along_axis(rankdata, 0, train_fmri)
            ranked_model = np.apply_along_axis(rankdata, 0, train_model)

            ranked_fmri -= ranked_fmri.mean(axis=0)
            ranked_model -= ranked_model.mean(axis=0)

            fmri_std = np.linalg.norm(ranked_fmri, axis=0)
            model_std = np.linalg.norm(ranked_model, axis=0)

            valid_voxels = fmri_std > 0
            valid_features = model_std > 0

            if not np.any(valid_voxels) or not np.any(valid_features):
                print(f"Warning: No valid voxels or model features in run {i}")
                voxel_ranks[i, :] = np.zeros(num_voxels)
                continue

            cov = ranked_fmri[:, valid_voxels].T @ ranked_model[:, valid_features]
            fmri_std = fmri_std[valid_voxels]
            model_std = model_std[valid_features]

            correlation_matrix = cov / (fmri_std[:, None] * model_std[None, :])

            if correlation_matrix.size == 0:
                print(f"Warning: Empty correlation matrix in run {i}")
                avg_corrs = np.zeros(valid_voxels.sum())
            else:
                avg_corrs = np.nanmean(correlation_matrix, axis=1)

            # Create full-size average correlation vector
            full_avg_corrs = np.full(num_voxels, -np.inf)
            full_avg_corrs[np.where(valid_voxels)] = avg_corrs

            voxel_ranks[i, :] = rankdata(-full_avg_corrs, method='average')

    average_ranks = np.mean(voxel_ranks, axis=0)
    top_voxel_count = int(0.1 * num_voxels)
    top_voxel_indices = np.argsort(average_ranks)[:top_voxel_count]

    return top_voxel_indices


def load_and_split_trimmed(path_or_array, run_start_indices, trim_start=3, trim_end=3, is_path=True):
    """
    Load a model from file or use the provided array,
    split it into runs, and trim edges (extra 1 at start for first run).
    """
    data = np.loadtxt(path_or_array, delimiter=",") if is_path else path_or_array
    runs = split_runs(data, run_start_indices, 0)
    trimmed_runs = [runs[0][trim_start + 1 : -trim_end]] + [r[trim_start:-trim_end] for r in runs[1:]]
    return trimmed_runs


In [5]:
#Loading all the foundation models and semantic models

# The foundation models that are compared to the fmri data
base_dirs = [
    "/Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/CLIPmultilingualmulti",
    "/Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/CLIPmultilingualtext",
    "/Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/XLM-roberta",
    "/Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/Llama"
]

# Dictionary to store all L1 similarity scores of the models
models = {}

# --- Process the renamed LLM model files ---
for base_dir in base_dirs:
    for file in os.listdir(base_dir):
        if file.endswith(".txt") and ("embedding+" in file or "layers" in file):
            file_path = os.path.join(base_dir, file)
            print(f"Processing: {file_path}")

            try:
                data = load_and_split_trimmed(file_path, run_start_indices)
                model_label = os.path.splitext(file)[0]
                models[model_label] = data
                print(f"Loaded: {model_label}")
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

# semantic models
try:
    # models["editing"] = load_and_split_trimmed(
    #     "/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/editing_4pcs_conv.1D",
    #     run_start_indices
    # )

    # models["acoustic"] = load_and_split_trimmed(
    #     np.hstack([
    #     np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundenvelope_8pcs_conv.1D", delimiter=","),
    #     np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundpowerspectrum_5pcs_conv.1D", delimiter=",")
    #         ]), run_start_indices, is_path=False)
    
    # models["visual"] = load_and_split_trimmed(
    #     np.hstack([
    #     np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_videogist_22pcs_conv.1D", delimiter=","),
    #     np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_videomotion_398pcs_conv.1D", delimiter=",")
    #         ]), run_start_indices, is_path=False)

    models["binder_abstractness"] = load_and_split_trimmed(
        "/Volumes/SSD-1TB/UU - thesis/fMRI data/conv_models_1614/binder_abstractness_conv.1D",
        run_start_indices
    )

    models["binder_concreteness"] = load_and_split_trimmed(
        "/Volumes/SSD-1TB/UU - thesis/fMRI data/conv_models_1614/binder_concreteness_conv.1D",
        run_start_indices
    )

    # models["word2vec"] = load_and_split_trimmed(
    #     "/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/highlevel_word2vec_72pcs_conv.1D",
    #     run_start_indices
    # )

except Exception as e:
    print(f"Error loading additional models: {e}")


Processing: /Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/CLIPmultilingualmulti/CLIPmultilingualmulti_layers11-15.txt
Loaded: CLIPmultilingualmulti_layers11-15
Processing: /Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/CLIPmultilingualtext/CLIPmultilingualtext_layers11-15.txt
Loaded: CLIPmultilingualtext_layers11-15
Processing: /Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/XLM-roberta/XLM-roberta_layers11-15.txt
Loaded: XLM-roberta_layers11-15
Processing: /Volumes/SSD-1TB/thesis paper/data/LLM embeddings/used LLM embeddings/Llama/Llama_layers_layers7-11.txt
Loaded: Llama_layers_layers7-11


In [11]:
#defining all brain area masks

ROIs = list(range(2, 6)) + ["body", "object", "scene", "face"]

language_mask = nib.load("/Volumes/SSD-1TB/UU - thesis/ROIs/allParcels-language-SN220.nii")
scene_mask = nib.load("/Volumes/SSD-1TB/UU - thesis/ROIs/ventralparcels/cvs_scene_parcels/fROIs-fwhm_5-0.0001.nii")
body_mask = nib.load("/Volumes/SSD-1TB/UU - thesis/ROIs/ventralparcels/cvs_body_parcels/fROIs-fwhm_5-0.0001.nii")
object_mask = nib.load("/Volumes/SSD-1TB/UU - thesis/ROIs/ventralparcels/cvs_object_parcels/fROIs-fwhm_5-0.0001.nii")
face_mask = nib.load("/Volumes/SSD-1TB/UU - thesis/ROIs/ventralparcels/cvs_face_parcels/fROIs-fwhm_5-0.0001.nii")

# Create dictionary
roi_masks = {
    **{i: language_mask for i in range(2, 6)},  # ROIs 2–5 all map to language_mask (where ROI 1 and 2 are taken together)
    "body": body_mask,
    "object": object_mask,
    "scene": scene_mask,
    "face": face_mask
}

In [12]:
ctrlAV = [12, 13, 14, 15, 16, 17, 18, 19, 22, 32]
ctrlA = [3, 4, 5, 6, 7, 8, 9, 10, 11, 27]
blind = [33, 35, 36, 38, 39, 41, 42, 43, 53]

groups = {'blind': blind
          ,'ctrlA': ctrlA
          ,'ctrlAV': ctrlAV}

In [ ]:
newcorrelations = True
storing_results = "results"

In [ ]:
#This piece of code computes L1 scores of the foundation models and semantic models, which will be compared to the fmri data

model_corr_path = storing_results + "/model_correlations.pkl"

if os.path.exists(model_corr_path) and newcorrelations is False:
    print("Loading existing model correlations")
    with open(model_corr_path, 'rb') as f:
        model_correlations = pickle.load(f)
else:
    print("Computing model correlations")
    model_correlations = {}

    for model_name, model_list in models.items():
        per_run = []

        for i in range(len(model_list)):
            test_model = model_list[i]
            L2b = []

            num_timepoints = test_model.shape[0]

            for t1 in range(num_timepoints):
                for t2 in range(t1 + 1, num_timepoints):
                    if test_model.ndim == 1:
                        # 1D: Use negative absolute distance
                        dist = -abs(test_model[t1] - test_model[t2])
                        L2b.append(dist)
                    else:
                        # 2D: Use Spearman correlation
                        corr = spearmanr(test_model[t1, :], test_model[t2, :]).correlation
                        L2b.append(corr)

            per_run.append(L2b)

        model_correlations[model_name] = per_run

    with open(model_corr_path, 'wb') as f:
        pickle.dump(model_correlations, f)


# loading the editing, sound and surprisal models

acoustic_full = np.hstack([
    np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundenvelope_8pcs_conv.1D", delimiter=","),
    np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundpowerspectrum_5pcs_conv.1D", delimiter=",")
])

editing = load_and_split_trimmed("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/editing_4pcs_conv.1D", run_start_indices)
surprisal = load_and_split_trimmed("/Volumes/SSD-1TB/UU - thesis/fMRI data/surprisal", run_start_indices)
acoustic = load_and_split_trimmed(acoustic_full, run_start_indices, is_path=False)


#this is the actual RSA

for participantgroup, ids in groups.items():
    print(f"Processing participant group {participantgroup}")
    results_dict = defaultdict(lambda: defaultdict(dict))
    best_voxels_dict = defaultdict(lambda: defaultdict(dict))

    # Fill in only `participantgroup`, keep {subj_id:03d} as a template
    base_path_template = f"/Volumes/SSD-1TB/UU - thesis/fMRI data/{participantgroup}/sub-{{subj_id:03d}}.nii.gz"

    for participant_id in ids:
        print(f"\tProcessing participant nr.{participant_id}")
        fmri_path = base_path_template.format(subj_id=participant_id)
        fmri_img = nib.load(fmri_path)

        # Loop over ROIs
        for roi, mask in roi_masks.items():
            print(f"\t\tProcessing roi {roi}")

            masked_data = mask_fmri_data(fmri_img, mask, roi)
            fmri_list = [(runs := split_runs(masked_data, run_start_indices, 0))[0][4:-3]] + [run[3:-3] for run in runs[1:]]

            for model_name, model_list in models.items():
                if model_list[0].ndim == 1:
                    spearman = False
                else:
                    spearman = True

                print(f"\t\t\tProcessing model {model_name}")

                L1 = [] # Store per-participant, per roi, per model correlations
                BV = [] # Store per participant, per roi, per model best voxels

                confound_list = [np.hstack([editing[i], acoustic[i], surprisal[i]]) for i in range(len(fmri_list))]
                residualized_fmri_list = cross_validated_residualize(fmri_list, confound_list)

                #Each run becomes a test set once
                for i in range(len(fmri_list)):
                    print(f"\t\t\t\tRun nr.{i} is now the testset")
                    
                    # Split residualized data
                    train_fmri = [residualized_fmri_list[j] for j in range(len(fmri_list)) if j != i]
                    train_model = [model_list[j] for j in range(len(model_list)) if j != i]

                    # Feature selection on residualized training data
                    best_voxels = return_ranks_voxels(train_fmri, train_model)
                    BV.append(best_voxels)

                    # Apply to residualized test run
                    test_fmri = residualized_fmri_list[i][:, best_voxels]

                    # First-order similarity of residualized fMRI data
                    fmri_corr = first_order_similarity(test_fmri)


                    #first order similarity model --> this only needs to be done once for each model (not per participant, roi and group)
                    model_corr = model_correlations[model_name][i] #L2b

                    second_order_corr = spearmanr(fmri_corr, model_corr).correlation
                    print(f"\t\t\t\t\tSecond order correlation: {second_order_corr}")

                    L1.append(second_order_corr)

                L0 = np.mean(L1)
                results_dict[participant_id][roi][model_name] = L0
                best_voxels_dict[participant_id][roi][model_name] = BV


    results_path = storing_results+f'/results_{participantgroup}.pkl'
    best_voxels_path = storing_results+f'/best_voxels_{participantgroup}.pkl'

    # Save results_dict
    with open(results_path, 'wb') as f:
        pickle.dump(dict(results_dict), f)

    # Save best_voxels_dict
    with open(best_voxels_path, 'wb') as f:
        pickle.dump(dict(best_voxels_dict), f)



In [ ]:
from sklearn.linear_model import LinearRegression

def residualize_model_data(model_data_list, surprisal_list):
    """
    Residualizes each model run with respect to surprisal.
    model_data_list: list of np arrays (1D or 2D)
    surprisal_list: list of np arrays (1D)
    Returns: residualized model data list
    """
    residualized = []

    for model_run, surprisal_run in zip(model_data_list, surprisal_list):
        if model_run.ndim == 1:
            X = surprisal_run.reshape(-1, 1)
            y = model_run
            reg = LinearRegression().fit(X, y)
            y_pred = reg.predict(X)
            residualized.append(y - y_pred)
        else:
            # 2D: residualize each feature
            X = surprisal_run.reshape(-1, 1)
            Y = model_run
            Y_resid = np.zeros_like(Y)
            for col in range(Y.shape[1]):
                reg = LinearRegression().fit(X, Y[:, col])
                Y_resid[:, col] = Y[:, col] - reg.predict(X)
            residualized.append(Y_resid)

    return residualized


In [ ]:
acoustic_full = np.hstack([
    np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundenvelope_8pcs_conv.1D", delimiter=","),
    np.loadtxt("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/lowlevel_soundpowerspectrum_5pcs_conv.1D", delimiter=",")
])

editing = load_and_split_trimmed("/Volumes/SSD-1TB/UU - thesis/fMRI data/Full_Models/Original/editing_4pcs_conv.1D", run_start_indices)
surprisal = load_and_split_trimmed("/Volumes/SSD-1TB/UU - thesis/fMRI data/convolved_surprisal", run_start_indices)
acoustic = load_and_split_trimmed(acoustic_full, run_start_indices, is_path=False)

In [ ]:
import os
import pickle
from scipy.stats import spearmanr

model_corr_path = storing_results + "/model_correlations.pkl"

if os.path.exists(model_corr_path) and newcorrelations is False:
    print("Loading existing model correlations")
    with open(model_corr_path, 'rb') as f:
        model_correlations = pickle.load(f)
else:
    print("Computing model correlations")
    model_correlations = {}

    for model_name, model_list in models.items():
        per_run = []

        for i in range(len(model_list)):
            test_model = model_list[i]
            L2b = []

            num_timepoints = test_model.shape[0]

            for t1 in range(num_timepoints):
                for t2 in range(t1 + 1, num_timepoints):
                    if test_model.ndim == 1:
                        # 1D: Use negative absolute distance
                        dist = -abs(test_model[t1] - test_model[t2])
                        L2b.append(dist)
                    else:
                        # 2D: Use Spearman correlation
                        corr = spearmanr(test_model[t1, :], test_model[t2, :]).correlation
                        L2b.append(corr)

            per_run.append(L2b)

        model_correlations[model_name] = per_run

    with open(model_corr_path, 'wb') as f:
        pickle.dump(model_correlations, f)

for participantgroup, ids in groups.items():
    print(f"Processing participant group {participantgroup}")
    results_dict = defaultdict(lambda: defaultdict(dict))
    best_voxels_dict = defaultdict(lambda: defaultdict(dict))

    # Fill in only `participantgroup`, keep {subj_id:03d} as a template
    base_path_template = f"/Volumes/SSD-1TB/UU - thesis/fMRI data/{participantgroup}/sub-{{subj_id:03d}}.nii.gz"

    for participant_id in ids:
        print(f"\tProcessing participant nr.{participant_id}")
        fmri_path = base_path_template.format(subj_id=participant_id)
        fmri_img = nib.load(fmri_path)

        # Loop over ROIs
        for roi, mask in roi_masks.items():
            print(f"\t\tProcessing roi {roi}")

            masked_data = mask_fmri_data(fmri_img, mask, roi)
            fmri_list = [(runs := split_runs(masked_data, run_start_indices, 0))[0][4:-3]] + [run[3:-3] for run in runs[1:]]

            for model_name, model_list in models.items():
                if model_list[0].ndim == 1:
                    spearman = False
                else:
                    spearman = True

                print(f"\t\t\tProcessing model {model_name}")

                L1 = [] # Store per-participant, per roi, per model correlations
                BV = [] # Store per participant, per roi, per model best voxels

                confound_list = [np.hstack([editing[i], acoustic[i], surprisal[i].reshape(-1, 1)]) for i in range(len(fmri_list))]
                residualized_fmri_list = cross_validated_residualize(fmri_list, confound_list)

                #Each run becomes a test set once
                for i in range(len(fmri_list)):
                    print(f"\t\t\t\tRun nr.{i} is now the testset")
                    
                    # Split residualized data
                    train_fmri = [residualized_fmri_list[j] for j in range(len(fmri_list)) if j != i]
                    train_model = [model_list[j] for j in range(len(model_list)) if j != i]

                    # Feature selection on residualized training data
                    best_voxels = return_ranks_voxels(train_fmri, train_model)
                    BV.append(best_voxels)

                    # Apply to residualized test run
                    test_fmri = residualized_fmri_list[i][:, best_voxels]

                    # First-order similarity of residualized fMRI data
                    fmri_corr = first_order_similarity(test_fmri)


                    #first order similarity model --> this only needs to be done once for each model (not per participant, roi and group)
                    model_corr = model_correlations[model_name][i] #L2b

                    second_order_corr = spearmanr(fmri_corr, model_corr).correlation
                    print(f"\t\t\t\t\tSecond order correlation: {second_order_corr}")

                    L1.append(second_order_corr)

                L0 = np.mean(L1)
                results_dict[participant_id][roi][model_name] = L0
                best_voxels_dict[participant_id][roi][model_name] = BV


    results_path = storing_results+f'/results_{participantgroup}.pkl'
    best_voxels_path = storing_results+f'/best_voxels_{participantgroup}.pkl'

    # Save results_dict
    with open(results_path, 'wb') as f:
        pickle.dump(dict(results_dict), f)

    # Save best_voxels_dict
    with open(best_voxels_path, 'wb') as f:
        pickle.dump(dict(best_voxels_dict), f)

